In [ ]:
!pip install -q openai anthropic cohere scikit-learn scipy matplotlib pandas supabase requests langchain_openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 6.8 MB/s eta 0:00:00


In [ ]:
import openai
import anthropic
import cohere
import requests
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from scipy.stats import f_oneway
from supabase import create_client
import os
import time
from datetime import datetime
from google.colab import files
from IPython.display import display, HTML, clear_output
from langchain_openai import OpenAIEmbeddings  # New package as per deprecation warning
import seaborn as sns

In [ ]:
#REading the API key
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OpenAI')
supabase_key = userdata.get('Supabase_key') #vector store
MISTRAL_API_KEY = userdata.get('Mistral')
CLAUDE_API_KEY = userdata.get('Anthropic')
COHERE_API_KEY = userdata.get('Cohere')
DEEPSEEK_API_KEY = userdata.get('Deepseek_new')
GEMINI_API_KEY = userdata.get('Gemini')

In [ ]:
 ### Supabase Credentials
SUPABASE_URL = "https://rrjbrtbsvdoxndchvchq.supabase.co"
SUPABASE_KEY = supabase_key

# Initialize API clients
openai.api_key = OPENAI_API_KEY
anthropic_client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)
cohere_client = cohere.Client(COHERE_API_KEY)

# Create Supabase client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [ ]:

# @title ⚙️ Step 4: Configure Experiment
# @markdown Select which models to include in your experiment:

use_claude = True # @param {type:"boolean"}
use_openai = True # @param {type:"boolean"}
use_mistral = True # @param {type:"boolean"}
use_cohere = True # @param {type:"boolean"}
use_deepseek = True # @param {type:"boolean"}

# @markdown Define the test questions:
question_text = "What is the Cafeteria plan?" # @param {type:"string"}
question_list = [

]

num_iterations = 1 # @param {type:"slider", min:1, max:50, step:1}

# @markdown Advanced settings:
top_k_docs = 2 # @param {type:"slider", min:1, max:10, step:1}
temperature = 0.7 # @param {type:"slider", min:0, max:1, step:0.1}
match_threshold = 0.5 # @param {type:"slider", min:0, max:1, step:0.1}

# Define LLM models
llm_models = {}

if use_claude:
    llm_models["Claude"] = {"provider": "anthropic", "model": "claude-3-7-sonnet-latest"}
if use_openai:
    llm_models["GPT"] = {"provider": "openai", "model": "gpt-4o"}
if use_mistral:
    llm_models["Mistral"] = {"provider": "mistral", "model": "mistral-large-latest"}
if use_cohere:
    llm_models["Cohere"] = {"provider": "cohere", "model": "command-a-03-2025"}
if use_deepseek:
    llm_models["Deepseek"] = {"provider": "deepseek", "model": "deepseek-chat"}

In [ ]:
def get_embedding(text):
    """Get embeddings for a text using OpenAI's embedding model"""
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,model="text-embedding-3-small")  # Use the appropriate model
    response = embeddings.embed_query(text)  # Correctly call the method to generate embeddings
    embedding = response
    return embedding


In [ ]:
# Generate the list of test questions
questions = question_list * num_iterations

In [ ]:
questions

In [ ]:
# @title 🔍 Step 5: Vector Search Function
def retrieve_context(query, top_k=top_k_docs):
    """Retrieve relevant documents from Supabase vector store using OpenAI embeddings and RPC with filter support"""
    try:
        # Get embedding for the query using OpenAI
        query_embedding = get_embedding(query)

        # Call the custom Supabase function via RPC with filter
        response = supabase.rpc(
            'match_documents',
            {
                'query_embedding': query_embedding,
                'match_count': top_k,
                'filter': {}  # You can pass filters here later if needed
            }
        ).execute()

        if not response.data or len(response.data) == 0:
            return "No relevant context found in the knowledge base."

        # Format retrieved documents as context
        context = "Context from knowledge base:\n\n"
        for i, doc in enumerate(response.data):
            content = doc.get('content', '')
            source = doc.get('source', 'Unknown source')
            context += f"Document {i+1}: {content}\nSource: {source}\n\n"

        return context

    except Exception as e:
        print(f"Error retrieving from vector store: {e}")
        return "Error retrieving context from knowledge base."


In [ ]:
#sanity check
query = "How can I access shared mailbox?"
context = retrieve_context(query)
print(context)

In [ ]:
# @title 🧠 Step 6: Query LLM with RAG for Different Providers
def query_llm_with_rag(llm_config, query):
    provider = llm_config["provider"]
    model = llm_config["model"]

    # Retrieve relevant context from Supabase
    context = retrieve_context(query)

    # Construct RAG prompt with retrieved context
    rag_prompt = f"""
{context}

Based on the above context from our knowledge base, please answer the following question:
{query}

If the context doesn't contain relevant information to answer the question,
please say so and answer based on your general knowledge.
"""

    try:
        # Call the appropriate API based on provider
        if provider == "openai":
            # Updated OpenAI client API call for v1.0+
            response = openai.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": rag_prompt}],
                temperature=temperature
            )
            return response.choices[0].message.content.strip()

        elif provider == "anthropic":
            response = anthropic_client.messages.create(
                model=model,
                messages=[{"role": "user", "content": rag_prompt}],
                max_tokens=1000,
                temperature=temperature
            )
            return response.content[0].text

        elif provider == "cohere":
            response = cohere_client.chat(
                message=rag_prompt,
                model=model,
                temperature=temperature
            )
            return response.text

        elif provider == "mistral":
            headers = {
                "Content-Type": "application/json",
                "Accept": "application/json",
                "Authorization": f"Bearer {MISTRAL_API_KEY}"
            }
            payload = {
                "model": model,
                "messages": [{"role": "user", "content": rag_prompt}],
                "temperature": temperature
            }
            response = requests.post(
                "https://api.mistral.ai/v1/chat/completions",
                headers=headers,
                json=payload
            )
            return response.json()["choices"][0]["message"]["content"].strip()

        elif provider == "deepseek":
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {DEEPSEEK_API_KEY}"
            }
            payload = {
                "model": model,
                "messages": [{"role": "user", "content": rag_prompt}],
                "temperature": temperature
            }
            response = requests.post(
                "https://api.deepseek.com/v1/chat/completions",
                headers=headers,
                json=payload
            )
            return response.json()["choices"][0]["message"]["content"].strip()

        else:
            return f"Error: Unsupported provider {provider}"

    except Exception as e:
        return f"Error with {provider} ({model}): {str(e)}"

In [ ]:
#Sanity checks
query = "What is a cafeteria plan?"
response = query_llm_with_rag(llm_models["Deepseek"], query)
print(response)

Based on the provided context, a **Cafeteria Plan** is an employee benefits program that allows employees to customize their compensation package by choosing from a menu of benefits tailored to their individual needs and preferences.  

### Key Features of the Cafeteria Plan at DTSC:  
1. **Flexibility** – Employees can select benefits that align with their personal and financial goals rather than a one-size-fits-all approach.  
2. **Options Available** – Choices include pension savings, IT device coverage, hospitalization insurance, extra vacation days, gym memberships, and more.  
3. **Annual Selection Process** – Around **November 25th**, employees receive a document listing options and have **three weeks** to submit their preferences.  
4. **Advantages** – The plan promotes empowerment, work-life balance, financial planning, and job satisfaction while aiding in talent retention.  

This system is designed to give employees control over their benefits, ensuring they receive what mat

In [ ]:

def run_experiment():
    # Track unique questions and their repetitions
    unique_questions = {}  # Maps question text to question base ID
    question_counts = {}   # Tracks how many times each unique question has been asked

    response_logs_by_question = {}
    df_data = []

    for i, question in enumerate(questions):
        # Check if this is a repetition of a previous question
        if question in unique_questions:
            base_question_id = unique_questions[question]
            question_counts[question] += 1
            repetition_num = question_counts[question]
            question_id = f"{base_question_id}_{repetition_num}"  # e.g., "q1_2" for second repetition
        else:
            # First time seeing this question
            base_question_id = f"q{len(unique_questions) + 1}"
            unique_questions[question] = base_question_id
            question_counts[question] = 1
            repetition_num = 1
            question_id = f"{base_question_id}_{repetition_num}"  # e.g., "q1_1" for first instance

        print(f"\n Iteration {i+1}/{len(questions)} — Question: {question} (ID: {question_id})")

        # Initialize the entry for this question instance
        response_logs_by_question[question_id] = {
            "question_text": question,
            "base_question_id": base_question_id,
            "repetition_num": repetition_num,
            "responses": {llm: [] for llm in llm_models}
        }

        for llm_name, config in llm_models.items():
            print(f"\n {llm_name} is generating a response...")
            try:
                response = query_llm_with_rag(config, question)
                #print(f" {llm_name} says:\n{response}\n")

                # Store the response in the nested structure
                response_logs_by_question[question_id]["responses"][llm_name].append(response)

                # Add row to DataFrame data
                df_data.append({
                    "question_id": question_id,
                    "base_question_id": base_question_id,
                    "repetition_num": repetition_num,
                    "question_text": question,
                    "llm": llm_name,
                    "response": response,
                    "iteration": i+1
                })

                # Add a small delay to avoid rate limits
                time.sleep(1)

            except Exception as e:
                print(f" Error with {llm_name}: {e}")

    # Create DataFrame from collected data
    response_df = pd.DataFrame(df_data)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"llm_rag_drift_log_{timestamp}.csv"
    response_df.to_csv(filename, index=False)
    print(f"\n✅ Log saved to '{filename}'")

    # For Google Colab: Allow downloading the file
    try:
        files.download(filename)
    except:
        print(f"To download the log file, use the Files panel on the left sidebar.")

    return response_logs_by_question, response_df

In [ ]:

response_logs, response_df = run_experiment()

Streaming output truncated to the last 5000 lines.

 Mistral is generating a response...

 Cohere is generating a response...

 Deepseek is generating a response...

 Iteration 85/500 — Question: How do I create a teams meeting? (ID: q5_9)

 Claude is generating a response...

 GPT is generating a response...

 Mistral is generating a response...

 Cohere is generating a response...

 Deepseek is generating a response...

 Iteration 86/500 — Question: What activities are included in the DTeam Spirit Challenge? (ID: q6_9)

 Claude is generating a response...

 GPT is generating a response...

 Mistral is generating a response...

 Cohere is generating a response...

 Deepseek is generating a response...

 Iteration 87/500 — Question: Can you tell me about Officient employee self-service? (ID: q7_9)

 Claude is generating a response...

 GPT is generating a response...

 Mistral is generating a response...

 Cohere is generating a response...

 Deepseek is generating a response...

 Iter

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
response_df

In [ ]:
response_df.to_csv("Hallucination Bias.csv")